# Time Series Ordinal Classification (TSOC) methods

In many TSC problems the categories follow an order relationship between them, classifiers taking into account this order can vastly improve the
performance achieved by nominal classifiers. Hence, Time Series Ordinal Classification (TSOC) is the field covering this gap, yet unexplored, in the literature. TSOC methods aims to include the categories order information during learning, thus boosting the convergence process.

Many UCR problems are ordinal in nature, e.g. the *EthanolConcentration* dataset, in the next figure we can visualize the ordinal information along the time series belonging to different classes of a given patter of this datasetn:

<img src="./img/ordinal-info-ethanol-concentration.png" width="550" alt="Pipeline classifier.">

### Setup

We load the *AtrialFibrillation* dataset, which is ordinal in nature, to test the methodologies.

In [1]:
from sklearn.metrics import mean_absolute_error

from aeon.datasets import load_atrial_fibrillation

X_train, y_train = load_atrial_fibrillation(split="train", return_X_y=True)
X_test, y_test = load_atrial_fibrillation(split="test", return_X_y=True)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(15, 2, 640) (15,) (15, 2, 640) (15,)


### Ordinal convolution-based methods

In the ordinal convolution-based methodologies the final set of features obtained from the kernel convolution and pooling operations is standardised and then fed to a LogisticAT classifier, which produces the final output. The convolution process remain unchanged from the nominal version.

<img src="./img/ordinal_convolution_based_arch.png" width="400" alt="Pipeline classifier.">

In [2]:
from aeon.classification.ordinal_classification import OrdinalRocketClassifier

methods = {
    "OrdinalRocketClassifier": OrdinalRocketClassifier(
        rocket_transform="rocket", num_kernels=100
    ),
    "OrdinalMiniRocketClassifier": OrdinalRocketClassifier(
        rocket_transform="minirocket", num_kernels=100
    ),
    "OrdinalMultiRocketClassifier": OrdinalRocketClassifier(
        rocket_transform="multirocket", num_kernels=100
    ),
}

for name, clf in methods.items():
    print("Method:", name)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(
        "-- Mean Absolute Error:",
        round(mean_absolute_error(y_test.astype(int), y_pred), 3),
        "\n",
    )

Method: OrdinalRocketClassifier
-- Mean Absolute Error: 1.067 

Method: OrdinalMiniRocketClassifier
-- Mean Absolute Error: 1.133 

Method: OrdinalMultiRocketClassifier
-- Mean Absolute Error: 0.867 



### Ordinal deep-learning methods

In ordinal deep learning techniques the activation layer is replaced by a Cummulative Link Model layer, which exploit the ordinal information of the data.

Ordinal InceptionTime Network architecture:

<img src="./img/ordinal_inceptiontime_arch.png" width="550" alt="Pipeline classifier.">

Ordinal ResNet architecture:

<img src="./img/ordinal_resnet_arch.png" width="550" alt="Pipeline classifier.">

In [4]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf

tf.get_logger().setLevel("ERROR")

from aeon.classification.ordinal_classification.deep_learning import (
    OrdinalInceptionTimeClassifier,
)

methods = {
    "OrdinalInceptionTime": OrdinalInceptionTimeClassifier(n_classifiers=2, n_epochs=30, save_best_model=False),
}

for name, clf in methods.items():
    print("Method:", name)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(
        "-- Mean Absolute Error:", mean_absolute_error(y_test.astype(int), y_pred), "\n"
    )

Method: OrdinalInceptionTime


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 530ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step
-- Mean Absolute Error: 1.1333333333333333 

